# Dashboard for the Argentinian Properties (II)
In this project, we revisit the [project on Argentinian Property listings](https://github.com/Mathhead/Sample_Projects/blob/master/argentinian_property_listings.ipynb) and build a simple dashboard. This time, we want to serve a regressor.

In [1]:
import numpy as np

np.random.seed( 1234567890 )

print( np.__version__ )

1.16.2


In [2]:
import pandas as pd

print( pd.__version__ )

0.24.2


In [3]:
property_data = pd.read_csv( "C:\\Users\\rhopi\\OneDrive\\Desktop\\kaggle data 1\\property listings data set\\ar_properties.csv" )

property_data.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,2019-09-15,-26.815439,-65.312393,Argentina,Tucumán,Yerba Buena,...,NaN,NaN,NaN,NaN,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,2019-09-15,-26.839469,-65.212790,Argentina,Tucumán,NaN,...,1.0,55.0,41.0,NaN,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.424820,-64.181225,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.421242,-64.190798,Argentina,Córdoba,Córdoba,...,1.0,48.0,48.0,NaN,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.426064,-64.180042,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal


In [4]:
property_data = property_data.dropna( subset = [ "price", "lon", "lat", "bathrooms", "surface_total" ] )

In [5]:
property_data = property_data[ property_data[ "currency" ] == "USD" ]

property_data = property_data[ property_data[ "property_type" ] == "Departamento" ]

property_data = property_data[ property_data[ "operation_type" ] == "Venta" ]

We drop all columns but the aforementioned.

In [6]:
for column in property_data.columns:
    
    if column not in [ "lat", "lon", "bathrooms", "surface_total", "price" ]:
        
        property_data = property_data.drop( [ column ], axis = 1 )
        
property_data.head()

,lat,lon,bathrooms,surface_total,price
61,-34.572445,-58.420624,3.0,193.0,935000.0
63,-34.586423,-58.414190,1.0,32.0,132900.0
64,-34.584758,-58.413319,1.0,88.0,230000.0
65,-34.578491,-58.431951,1.0,36.0,108000.0
68,-34.561679,-58.458689,1.0,80.0,269000.0


In [7]:
print( property_data.isna().sum() )

lat              0
lon              0
bathrooms        0
surface_total    0
price            0
dtype: int64


In [8]:
train_x = property_data.copy()

train_x = train_x.drop( [ "price" ], axis = 1 ).to_numpy()

train_y = property_data[ "price" ].to_numpy()

Now that the data are prepared, let's fit a random forest.

In [9]:
import sklearn
from sklearn.ensemble import RandomForestRegressor

print( sklearn.__version__ )

0.20.3


In [10]:
forest = RandomForestRegressor()

myforest = forest.fit( train_x, train_y )

C:\Users\rhopi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


And now, we serve the random forest regressor via dash.

In [11]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

print( dash.__version__ )

1.14.0


In [12]:
import jupyter_dash
from jupyter_dash import JupyterDash

print( jupyter_dash.__version__ )

0.3.1


In [13]:
import plotly
import plotly.graph_objs as go

print( plotly.__version__ )

4.9.0


In [14]:
app_predict = JupyterDash( __name__ )

In [15]:
app_predict.css.append_css( { "external_url" : "https://codepen.io/chriddyp/pen/bWLwgP.css" } )

In [16]:
input_lon = dcc.Input( id = "lon", type = "number", value = -60 )

input_lat = dcc.Input( id = "lat", type = "number", value = -35 )

input_bath = dcc.Input( id = "bath", type = "number", value = 1 )

input_surface = dcc.Input( id = "surface", type = "number", value = 100 )

In [17]:
div_lon = html.Div( children = [ html.H3( "Longitude: " ), input_lon ], className = "four columns" )

div_lat = html.Div( children = [ html.H3( "Latitude: " ), input_lat ], className = "four columns" )

div_bath = html.Div( children = [ html.H3( "Bathrooms: " ), input_bath ], className = "four columns" )

div_surface = html.Div( children = [ html.H3( "Surface total: " ), input_surface ], className = "four columns" )

In [18]:
def get_prediction( lon, lat, bath, surface ):
    
    data = np.array( [ lon, lat, bath, surface ] ).reshape( 1, -1 )
    
    pred = myforest.predict( data )[ 0 ]
    
    return pred

In [19]:
app_predict.layout = html.Div( [ html.H1( "Web App for the Prediction of Argentinian Property Prices" ),
                             html.H2( "Please enter the values for the Longitude, the Latitude, the number of bathrooms and the total covered surface" ),
                             html.Div( children = [ div_lon, div_lat, div_bath, div_surface ] ),
                             html.H1( id = "output", style = { "margin-top" : "50px", "text-align" : "center" } ) ] )

In [20]:
@app_predict.callback( Output( "output", "children" ), [ Input( "lon", "value" ), Input( "lat", "value" ), 
                                                       Input( "bath", "value" ), Input( "surface", "value" ) ] )

def show_prediction( lon, lat, bath, surface ):
    
    pred = get_prediction( lon, lat, bath, surface )
    
    return "The predicted property value is " + str( pred ) + "US$"

In [21]:
app_predict.run_server( mode = "inline" )

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.



C:\Users\rhopi\Anaconda3\lib\site-packages\dash\resources.py:68: UserWarning: You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources
  ).format(s["external_url"])
